In [2]:
import pandas as pd
import numpy as np
import csv

In [11]:
nfb = pd.read_csv('NeuroFB_part5.csv', low_memory=False)
nfb.head()

val = nfb.iloc[0].values
nfb=nfb.loc[1:,]

nfb.columns

ident_dict = dict(zip(nfb.columns.get_values(), val.T))

remove_keys = [] 
for key, value in ident_dict.items(): 
    if(type(value) == float or type(value) == np.float_):
        if(np.isnan(value)): 
            remove_keys.append(key) 
print(remove_keys)

for i in range(len(remove_keys)):
    if remove_keys[i] in ident_dict:
        del ident_dict[remove_keys[i]]

nfb.set_index('queried_ursi', inplace=True)

In [12]:
#Codebook reading
pre_cb = {}
codebook_lines = []
with open('CB.csv') as csvfile:
    csvreader = csv.reader(csvfile, dialect='excel')
    for row in csvreader:
        codebook_lines.append(row)

current_key = ''
for line_vals in codebook_lines:
    if 'Instrument' in line_vals[0]:
        continue
    
    if line_vals[3]:
        if line_vals[3] in pre_cb:
            raise ValueError('Key {0} already exists in dictionary!'.format(line_vals[3]))
        pre_cb[line_vals[3]] = {}
        current_key = line_vals[3]

    if not current_key:
        raise ValueError("Trying to set value with empty key")
    pre_cb[current_key][line_vals[5]] = line_vals[4]
    

In [13]:
#Instrument keys readin from codebook
instrument_dict = {}
current_instrmt = ''
previous_instrmt = ''
for line_vals in codebook_lines:
    if 'Instrument' in line_vals[0]:
        continue
    previous_instrmt = ''    
    if line_vals[0]:
        if line_vals[0] in instrument_dict:
            raise ValueError('Error')
        current_instrmt = line_vals[0]
        instrument_dict.setdefault(current_instrmt, [])
    while(current_instrmt!=previous_instrmt):
        if line_vals[3]:
            instrument_dict[current_instrmt].append(line_vals[3])
        previous_instrmt = current_instrmt

In [155]:
new_col = ['queried_ursi', 'visit_id']

for key in pre_cb:
    for c in nfb.columns:
        if ('_'+key+'_') in c:   
            nfb_col = c[(c.find('_'+key)+1):]
        elif (c[0]=='V'):
            if ((c[c.find('_')+1:])[0].isdigit()): #if first character after the first '_' is a digit
                nfb_col = c[c.find('_', c.find('_') + 1)+1:]
            elif (c[c.rfind('_', 0, c.rfind('_'))+1:])[0].isdigit():
                nfb_col = c[c.rfind('_', 0, c.rfind('_', 0, c.rfind(c[c.rfind('_', 0, c.rfind('_')):]))+1)+1:]
            else:
                nfb_col = c[c.rfind('_', 0, c.rfind('_'))+1:]
        new_col.append(nfb_col)
unique_list = [x for i, x in enumerate(new_col) if new_col.index(x) == i]
#unique_list

In [154]:
key = 'MRIQ'

c = 'V3_NFB3_MRIQ_01'
if ('_'+key) in c:   
    print( c[(c.find('_'+key)+1):] + 'AAAA')
elif (c[0]=='V'):
    if ((c[c.find('_')+1:])[0].isdigit()):
        print(c[c.find('_', c.find('_') + 1)+1:] )
    elif (c[c.rfind('_', 0, c.rfind('_'))+1:])[0].isdigit():
        print(c[c.rfind('_', 0, c.rfind('_', 0, c.rfind(c[c.rfind('_', 0, c.rfind('_')):]))+1)+1:] )
    else:
        print( c[c.rfind('_', 0, c.rfind('_'))+1:] )

MRIQ_01AAAA


In [156]:
(sorted(unique_list, reverse=True))

['visit_id',
 'queried_ursi',
 'RVIP_08',
 'RVIP_07',
 'RVIP_06',
 'RVIP_05',
 'RVIP_04',
 'RVIP_03',
 'RVIP_02',
 'RVIP_01',
 'RRS_026',
 'RRS_025',
 'RRS_024',
 'RRS_023',
 'RRS_022',
 'RRS_021',
 'RRS_020',
 'RRS_019',
 'RRS_018',
 'RRS_017',
 'RRS_016',
 'RRS_015',
 'RRS_014',
 'RRS_013',
 'RRS_012',
 'RRS_011',
 'RRS_010',
 'RRS_009',
 'RRS_008',
 'RRS_007',
 'RRS_006',
 'RRS_005',
 'RRS_004',
 'RRS_003',
 'RRS_002',
 'RRS_001',
 'PTQ_016',
 'PTQ_015',
 'PTQ_014',
 'PTQ_013',
 'PTQ_012',
 'PTQ_011',
 'PTQ_010',
 'PTQ_009',
 'PTQ_008',
 'PTQ_007',
 'PTQ_006',
 'PTQ_005',
 'PTQ_004',
 'PTQ_003',
 'PTQ_002',
 'PTQ_001',
 'PSQI_27',
 'PSQI_26',
 'PSQI_25',
 'PSQI_24',
 'PSQI_23',
 'PSQI_18',
 'PSQI_17',
 'PSQI_16',
 'PSQI_15',
 'PSQI_14',
 'PSQI_13',
 'PSQI_12',
 'PSQI_11',
 'PSQI_10e3',
 'PSQI_10e2',
 'PSQI_10e1',
 'PSQI_10d',
 'PSQI_10c',
 'PSQI_10b',
 'PSQI_10a',
 'PSQI_10',
 'PSQI_09',
 'PSQI_08',
 'PSQI_07',
 'PSQI_06',
 'PSQI_05j2',
 'PSQI_05j1',
 'PSQI_05i',
 'PSQI_05h',
 'PSQI

In [17]:
new_df = pd.DataFrame(columns=unique_list)
columns = nfb.columns.sort_values()

Index(['V1REP_2_NEFFI_23', 'V1REP_2_NEOFFI_01', 'V1REP_2_NEOFFI_02',
       'V1REP_2_NEOFFI_03', 'V1REP_2_NEOFFI_04', 'V1REP_2_NEOFFI_05',
       'V1REP_2_NEOFFI_06', 'V1REP_2_NEOFFI_07', 'V1REP_2_NEOFFI_08',
       'V1REP_2_NEOFFI_09',
       ...
       'visit_instance.43', 'visit_instance.44', 'visit_instance.45',
       'visit_instance.46', 'visit_instance.47', 'visit_instance.5',
       'visit_instance.6', 'visit_instance.7', 'visit_instance.8',
       'visit_instance.9'],
      dtype='object', length=4205)

In [18]:
def convertToCodebook(k, value):
    if(type(value)==str and value.lower()!='nan'):
        if k in pre_cb.keys():
            if value in pre_cb[k]:
                return pre_cb[k][value]
            elif value not in pre_cb[k]:
                return value 
        else:
            return value
    elif((type(value) == float or type(value) == np.float64) and np.isnan(value)):
        return value

In [19]:
#Dataframe restructuring WITH codebook converting
row_counter = -1
target_index_counter = -1
previous_visit = ''

for i in range(len(nfb.index)):
    previous_visit = ''
    row_counter+=1
    for c in columns:
        for key in unique_list:
            if key in c:
                visit_col = c[:(c.find(key))-1]
                nfb_3_col = c[c.find(key):]
                if (previous_visit != visit_col):
                    target_index_counter+=1
                    new_df.at[target_index_counter, 'visit_id'] = visit_col
                    new_df.at[target_index_counter, 'queried_ursi'] = nfb.index[row_counter]
                    previous_visit = visit_col
                val = nfb.loc[nfb.index[row_counter],c]
                new_df.at[target_index_counter, nfb_3_col] = convertToCodebook(key, val)


In [23]:
def drop_NanRows(df):
    drop_row = True
    drop_list = []
    for i in range(len(df.index)):
        row = df.iloc[i]
        drop_row = True
        for f in range(len(row)-2):
            if(type(row[f+2]) == float or type(row[f+2]) == np.float64):
                if not(np.isnan(row[f+2])):
                    drop_row = False
            elif(type(row[f+2]) == str and row[f+2].lower() != 'nan'):
                drop_row = False
            else:
                drop_row = False
        if(drop_row):
            drop_list.append(i)
    return drop_list

In [27]:
#Drop nan rows
drop_NanRows(new_df)
new_df = new_df.drop(drop_NanRows(new_df))
nfb_part = new_df.reset_index(drop=True)
nfb_part = nfb_part.dropna(axis=1, how='all')

In [121]:
def get_instrumentID(column_name, var_id):
    inst_id = ''
    found = False
    if not var_id:
        for inst, variables in instrument_dict.items():
            for val in variables:
                if(column_name == val):
                    inst_id = inst
                    found = True
    if (not found) or var_id:
        if((column_name[column_name.rfind('_', 0, column_name.rfind('_'))+1:][0]).isdigit()): #if first character after the last '_' is a digit
            inst_id = column_name[:(column_name.rfind('_', 0, column_name.rfind('_')))]
        else:
            inst_id = column_name[:(column_name.rfind('_'))]
        for inst, variables in instrument_dict.items():             
            if (inst_id == get_instrumentID(variables[0], not var_id)):
                inst_id = inst
    return inst_id

In [127]:
def create_instrumentDF(instrmt):
    #get column names
    a_columns = ['queried_ursi','visit_id']
    for c in nfb_part.columns:
        if instrmt in c:
            a_columns.append(c)
    df = pd.DataFrame(columns=a_columns)
    
    #add to dataframe
    for i in range(len(nfb_part.index)):
        for c in a_columns:
            if instrmt in c:
                df.at[i, 'queried_ursi'] = nfb_part.queried_ursi[i]
                df.at[i, 'visit_id'] = nfb_part.visit_id[i]
                df.at[i, c] = nfb_part.loc[i, c]
    #Drop nan rows
    df = df.drop(drop_NanRows(df))
    df.set_index('queried_ursi', inplace=True)
    return df

In [128]:
previous_instrument = ''
instrument = ''
instrumentDF_dict = {}
instrument_id = ''

for c in nfb_part.columns: 
    if(c != 'queried_ursi' and c != 'visit_id'):
        instrument = c[:(c.find('_'))] 
        instrument_id = get_instrumentID(c, False)
    if (previous_instrument != instrument):
        instrumentDF_dict[instrument_id] = create_instrumentDF(instrument)
        previous_instrument = instrument

column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrum

column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrum

column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrum

column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrum

column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrument Debrief
column added: Debrief_02|instrument Debrief
column added: Debrief_03|instrument Debrief
column added: Debrief_04|instrum